# BTBSession

## What is BTBSession

BTBSession provides a simplified user interface to be able to search the best solution for your tuning problem by combining tuners and selectors with as little steps required as possible.

## Creating a BTBSession

We will guide you through the necessary steps to get started using BTBSession
to select and tune the best model to solve a Machine Learning problem.

In particular, in this example we will be using ``BTBSession`` to perform solve the [Boston](
http://lib.stat.cmu.edu/datasets/boston) regression problem by selecting between the
`ExtraTreesRegressor` and the `RandomForestRegressor` models from [scikit-learn](
https://scikit-learn.org/) while also searching for their best Hyperparameter configuration.

### Prepare a scoring function

The first step in order to use the `BTBSession` class is to develop a scoring function.

This is a Python function that, given a model name and a hyperparameter configuration,
evaluates the performance of the model on your data and returns a score.

In [1]:
import warnings

from sklearn.datasets import load_boston
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.metrics import make_scorer, r2_score
from sklearn.model_selection import cross_val_score

warnings.filterwarnings('ignore')

dataset = load_boston()
models = {
    'random_forest': RandomForestRegressor,
    'extra_trees': ExtraTreesRegressor,
}

def scoring_function(model_name, hyperparameter_values):
    model_class = models[model_name]
    model_instance = model_class(**hyperparameter_values)
    scores = cross_val_score(
        estimator=model_instance,
        X=dataset.data,
        y=dataset.target,
        scoring=make_scorer(r2_score)
    )
    return scores.mean()

### Define the tunable hyperparameters

The second step is to define the hyperparameters that we want to tune for each model as `Tunables`.


In [2]:
from btb.tuning import Tunable
from btb.tuning.hyperparams import CategoricalHyperParam, IntHyperParam

tunables = {
    'random_forest': Tunable({
        'max_features': CategoricalHyperParam(choices=[None, 'auto', 'log2', 'sqrt']),
        'min_samples_split': IntHyperParam(min=2, max=20, default=2),
        'min_samples_leaf': IntHyperParam(min=1, max=20, default=2)
    }),
    'extra_trees': Tunable({
        'max_features': CategoricalHyperParam(choices=[None, 'auto', 'log2', 'sqrt']),
        'min_samples_split': IntHyperParam(min=2, max=20, default=2),
        'min_samples_leaf': IntHyperParam(min=1, max=20, default=2)
    })
}

### Create BTBSession instance

Once you have defined a scoring function and the tunable hyperparameters specification of your
models, you can create the instance of `btb.BTBSession`.

BTBSession accepts the following arguments:

- `tunables` (dict): Python dictionary that has as keys the name of the tunable and as value a dictionary with the tunable hyperparameters or an ``btb.tuning.tunable.Tunable`` instance.
- `scorer` (callable object / function): A callable object or function with signature ``scorer(tunable_name, config)`` wich should return only a single value.
- `tuner_class` (btb.tuning.tuner.BaseTuner): A tuner based on BTB ``BaseTuner`` class. This tuner will manage the new proposals. Defaults to ``btb.tuning.tuners.gaussian_process.GPTuner``
- `selector_class` (btb.selection.selector.Selector): A selector based on BTB ``Selector`` class. This will determinate which one of the tunables is performing better, and which one to test next. Defaults to ``btb.selection.selectors.ucb1.UCB1``
- `maximize` (bool): If ``True`` the scores are interpreted as bigger is better, if ``False`` then smaller is better, this should depend on the problem type (maximization or minimization). Defaults to ``True``.
- `max_erors` (int): Amount of errors allowed for a tunable to not generate a score. Once this amount of errors is reached, the tunable will be removed from the list. Defaults to 1.
- `verbose` (bool): If ``True`` a progress bar will be displayed for the ``run`` process.

For now all you need to do is pass the tunable hyperparameters scpecification and the scoring function.

In [3]:
from btb import BTBSession

session = BTBSession(
    tunables=tunables,
    scorer=scoring_function
)

## Using BTBSession

### Run

BTBSession works with it's main method called `run`. This method accepts as an argument
the amount of tuning iterations to perform. By default this argument is `None` wich means
that it will run until it's not stopped by the user or a `StopTuning` exception is raised.

For now you can call the `run` method indicating how many tunable iterations you want the
Session to perform:

In [ ]:
best_proposal = session.run(10)

### Exploring the result

The result will be a dictionary indicating the name of the best model that could be found
and the hyperparameter configuration that was used:

In [ ]:
best_proposal

The session object also contains this `best_proposal` as an attribute 

In [ ]:
session.best_proposal

### Resume session

The session object allows the user to resume the tuning process by calling again the `run` method:

In [ ]:
best_proposal = session.run(50)
best_proposal

## Fitting the best solution

One we have found the best possible solution, we are ready to learn a model from our data in order to make predictions.
To do this, we will have to retrieve from the `best_proposal` dict both the name and the configuration of the best solution.

In [ ]:
best_model_name = best_proposal['name']
hyperparameters = best_proposal['config']
best_model_class = models[best_model_name]
model_instance = best_model_class(**hyperparameters)

In [ ]:
model_instance.fit(dataset.data, dataset.target)